

**
Un objectif supervisé de régression:prédire les baisses de prix (avec  RandomForestRegressor)**





In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# Lire le fichier depuis l'environnement Colab
df = pd.read_excel("entreprise.xlsx")

# Afficher un aperçu des données
df.head()



/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,Produit,Marque,Prix,Prix_avant_remise,Taille,Pourcentage,magasin,Code_Produit,id_entreprise,Date_D_Achat,Quantite_Achetee
0,Boisson gazeuse,VIVA,2.10,2.49,1L,-16%,MG,M-1501,3,30/09/2023,3
1,Boisson gazeuse,TROPICO,2.10,2.49,1L,-16%,MG,M-1502,3,22/01/2023,4
2,Boisson gazeuse,RAOUA,2.10,2.49,1L,-16%,MG,M-1503,3,01/03/2024,4
3,Boisson gazeuse existe en plusieurs saveurs,VIVA,1.15,1.33,24 cl,-14%,MG,M-1504,3,23/01/2023,2
4,Boisson gazeuse,RAOUA,1.15,1.33,24 cl,-14%,MG,M-1505,3,31/05/2024,3


In [2]:
# Afficher les noms des colonnes du DataFrame
print(df.columns)


Index(['Produit', 'Marque', 'Prix', 'Prix_avant_remise', 'Taille',
       'Pourcentage', 'magasin', 'Code_Produit', 'id_entreprise',
       'Date_D_Achat', 'Quantite_Achetee'],
      dtype='object')


In [3]:
print(df[["Date_D_Achat", "Prix", "Prix_avant_remise"]].head())
print(df.dtypes)


  Date_D_Achat  Prix  Prix_avant_remise
0   30/09/2023  2.10               2.49
1   22/01/2023  2.10               2.49
2   01/03/2024  2.10               2.49
3   23/01/2023  1.15               1.33
4   31/05/2024  1.15               1.33
Produit               object
Marque                object
Prix                 float64
Prix_avant_remise    float64
Taille                object
Pourcentage           object
magasin               object
Code_Produit          object
id_entreprise          int64
Date_D_Achat          object
Quantite_Achetee       int64
dtype: object


In [4]:
df["Date_D_Achat"] = pd.to_datetime(df["Date_D_Achat"], errors='coerce')
df["Date_D_Achat"]

<ipython-input-4-c563ae313c96>:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["Date_D_Achat"] = pd.to_datetime(df["Date_D_Achat"], errors='coerce')


,Date_D_Achat
0,2023-09-30
1,2023-01-22
2,2024-03-01
3,2023-01-23
4,2024-05-31
...,...
167,2023-04-25
168,2023-04-25
169,2024-09-17
170,2023-11-03


In [5]:
print(df[["Date_D_Achat", "Prix", "Prix_avant_remise"]].isnull().sum())
print(df[["Prix", "Prix_avant_remise"]].describe())


Date_D_Achat          0
Prix                  3
Prix_avant_remise    19
dtype: int64
             Prix  Prix_avant_remise
count  169.000000         153.000000
mean     5.934852           7.287124
std      5.733860           7.606672
min      0.380000           0.430000
25%      2.290000           2.750000
50%      3.990000           5.190000
75%      6.490000           8.100000
max     29.990000          41.990000


In [6]:
df["Prix"] = df["Prix"].fillna(df["Prix"].mean())
df["Prix_avant_remise"] = df["Prix_avant_remise"].fillna(df["Prix_avant_remise"].mean())


In [7]:
# Imputer les valeurs manquantes sans utiliser inplace=True
df['Quantite_Achetee'] = df['Quantite_Achetee'].fillna(df['Quantite_Achetee'].mean())
df['Produit'] = df['Produit'].fillna(df['Produit'].mode()[0])
df['Marque'] = df['Marque'].fillna(df['Marque'].mode()[0])
df['Pourcentage'] = df['Pourcentage'].fillna(df['Pourcentage'].mode()[0])
# Vérifier les NaN après traitement
print("Après traitement des NaN:")
print(df.isna().sum())


Après traitement des NaN:
Produit               0
Marque                0
Prix                  0
Prix_avant_remise     0
Taille               65
Pourcentage           0
magasin               0
Code_Produit          0
id_entreprise         0
Date_D_Achat          0
Quantite_Achetee      0
dtype: int64


In [8]:
print(df['Pourcentage'])

0      -16%
1      -16%
2      -16%
3      -14%
4      -14%
       ... 
167    -25%
168    -25%
169    -25%
170    -25%
171    -25%
Name: Pourcentage, Length: 172, dtype: object


In [9]:
# Imputer les NaN dans la colonne Taille (avec le mode)
df['Taille'] = df['Taille'].fillna(df['Taille'].mode()[0])


# Vérification après traitement
print("Après traitement des NaN:")
print(df.isna().sum())

Après traitement des NaN:
Produit              0
Marque               0
Prix                 0
Prix_avant_remise    0
Taille               0
Pourcentage          0
magasin              0
Code_Produit         0
id_entreprise        0
Date_D_Achat         0
Quantite_Achetee     0
dtype: int64


In [10]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
# Initialiser LabelEncoder
label_encoder = LabelEncoder()

# Appliquer Label Encoding sur la colonne 'Produit'
df['Produit_encoded'] = label_encoder.fit_transform(df['Produit'])

# Afficher le DataFrame avec l'encodage
print(df[['Produit', 'Produit_encoded']])

                                          Produit  Produit_encoded
0                                 Boisson gazeuse                8
1                                 Boisson gazeuse                8
2                                 Boisson gazeuse                8
3    Boisson gazeuse existe en plusieurs saveurs                10
4                                Boisson gazeuse                 9
..                                            ...              ...
167                            GEL COIFFANT EFFET               66
168                               SAVON DE M?NAGE              112
169                               Savon de m?nage              116
170                             Vinaigre a m?nage              132
171                               Savon de m?nage              116

[172 rows x 2 columns]


In [11]:
df['Marque_encoded'] = label_encoder.fit_transform(df['Marque'])
df['Taille_encoded'] = label_encoder.fit_transform(df['Taille'])

In [12]:
# Convertir la colonne 'Date_D_Achat' en format datetime
df['Date_D_Achat'] = pd.to_datetime(df['Date_D_Achat'], format='%d/%m/%Y')


In [13]:
# Extraire des informations de date (année, mois, jour, etc.)
df['Annee'] = df['Date_D_Achat'].dt.year
df['Mois'] = df['Date_D_Achat'].dt.month
df['Jour'] = df['Date_D_Achat'].dt.day


In [17]:
# Convertir la colonne 'Pourcentage' en nombre (en enlevant le % et en divisant par 100 si nécessaire)
df['Pourcentage'] = df['Pourcentage'].replace({r'%': ''}, regex=True).astype(float)



print(df['Pourcentage'])


0     -16.0
1     -16.0
2     -16.0
3     -14.0
4     -14.0
       ... 
167   -25.0
168   -25.0
169   -25.0
170   -25.0
171   -25.0
Name: Pourcentage, Length: 172, dtype: float64


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Sélectionner les features et la cible
X = df[['Produit_encoded', 'Marque_encoded', 'Taille_encoded', 'Quantite_Achetee', 'Annee', 'Mois', 'Jour']]
y = df['Pourcentage']  # Cible

# Diviser en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalisation des features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)




In [24]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Entraîner le modèle de régression
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Prédictions sur les données de test
y_pred = model.predict(X_test)



In [25]:


import numpy as np
from sklearn.metrics import mean_squared_error

# Calcul de la MSE (Mean Squared Error)
mse = mean_squared_error(y_test, y_pred)

# Calcul du RMSE en prenant la racine carrée de la MSE
rmse = np.sqrt(mse)

print(f"RMSE : {rmse}")




RMSE : 5.62263715452354


In [26]:
from sklearn.metrics import r2_score

# Calcul du R²
r2 = r2_score(y_test, y_pred)

print(f"R² : {r2}")


R² : -0.3002689195541226


In [ ]:
df.head()

In [ ]:
#interpretation
#Un RMSE faible signifie que les valeurs prédites sont proches des valeurs réelles, donc le modèle est précis.
#Un RMSE élevé indique que les prédictions sont souvent loin des valeurs réelles, ce qui signifie que le modèle fait des erreurs importantes.

In [27]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

# Définir les paramètres à tester
param_grid = {
    'n_neighbors': [5, 10, 15, 20],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree'],
}

# Initialisation du modèle
knn = KNeighborsRegressor()

# Recherche des meilleurs hyperparamètres
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Affichage des meilleurs paramètres
print(f"Meilleurs paramètres : {grid_search.best_params_}")
print(f"Meilleur score : {grid_search.best_score_}")


Meilleurs paramètres : {'algorithm': 'auto', 'n_neighbors': 20, 'weights': 'uniform'}
Meilleur score : -51.00970701058201


In [28]:
from sklearn.model_selection import GridSearchCV
param_grid = {'n_neighbors': [3, 5, 7, 10], 'weights': ['uniform', 'distance']}
grid_search = GridSearchCV(KNeighborsRegressor(), param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train)
print("Meilleurs paramètres :", grid_search.best_params_)


Meilleurs paramètres : {'n_neighbors': 10, 'weights': 'distance'}


In [31]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Normaliser les données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Normalisation sur X_train
X_test_scaled = scaler.transform(X_test)  # Appliquer la même transformation sur X_test

# Créer et entraîner le modèle KNN avec les meilleurs paramètres
knn_best = KNeighborsRegressor(n_neighbors=20, weights='uniform')
knn_best.fit(X_train_scaled, y_train)

# Prédictions sur les données de test
y_pred_knn_best = knn_best.predict(X_test_scaled)

# Évaluation
rmse_knn_best = np.sqrt(mean_squared_error(y_test, y_pred_knn_best))
r2_knn_best = r2_score(y_test, y_pred_knn_best)

# Affichage des résultats
print(f"KNN (avec meilleurs paramètres) - RMSE: {rmse_knn_best:.2f},")
# Calcul du R² ajusté
n = len(y_test)  # Nombre d'observations dans les données de test
p = X_test_scaled.shape[1]  # Nombre de prédicteurs dans le modèle


r2_knn_adj = 1 - ((1 + r2_knn_best) * (n -1)) / (n + p + 1)

print(f"KNN - RMSE: {rmse_knn_best:.2f}, R² : {r2_knn_adj:.2f}")


KNN (avec meilleurs paramètres) - RMSE: 5.31,
KNN - RMSE: 5.31, R² : 0.34


In [32]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score

# Créer un modèle XGBoost
xgb_model = xgb.XGBRegressor(random_state=42, objective='reg:squarederror')

# Entraîner le modèle
xgb_model.fit(X_train_scaled, y_train)

# Prédire sur les données de test
y_pred_xgb = xgb_model.predict(X_test_scaled)

# Calculer les performances
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
r2_xgb = r2_score(y_test, y_pred_xgb)

n = len(y_test)  # Nombre d'observations dans les données de test
p = X_test_scaled.shape[1]  # Nombre de prédicteurs dans le modèle
n
print(n,p)
r2_xgb_adj = 1 +((1 + r2_xgb) * (n -1)) / (n - p - 1)

print(f"XGBoost - RMSE: {rmse_xgb:.2f}, R² : {r2_xgb_adj:.2f}")


35 7
XGBoost - RMSE: 7.16, R² : 0.86



# **Série temp**

